In [1]:
import sys
import warnings
if not sys.warnoptions:
  warnings.simplefilter("ignore")

In [ ]:
! pip install deap
! pip install scoop

In [3]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn.model_selection as model_selection
import pickle, scipy, csv, statistics, math, warnings, joblib, random, numpy
from sklearn import metrics, svm, datasets
from math import log,e, floor
from time import time
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score,cross_val_predict,GridSearchCV, train_test_split, StratifiedKFold, KFold, cross_validate, learning_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import plot_confusion_matrix, mean_absolute_error,accuracy_score,r2_score,confusion_matrix ,classification_report, f1_score, precision_score, recall_score
from sklearn import metrics, preprocessing, svm
from sklearn.linear_model import LogisticRegression
from deap import creator, base, tools, algorithms
from scoop import futures
from sklearn.utils import shuffle
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def emotion_label(labels, class_label):
	em_labels = []
	if(class_label == "valence"):
		for i in range(0, labels.shape[0]):
			if (labels[i][0]>5): # high valence
				em_labels.append(1)
			else: # low valence
				em_labels.append(0)
		return em_labels
	elif(class_label == "arousal"):
		for i in range(0, labels.shape[0]):
			if (labels[i][1]>5): # high arousal
				em_labels.append(1)
			else: # low arousal
				em_labels.append(0)
		return em_labels
	elif(class_label == "all"):
		for i in range(0, labels.shape[0]):
			if (labels[i][0]>5): # high valence
				if(labels[i][1]>5): # high arousal
					em_labels.append(1) # HVHA
				else:
					em_labels.append(0) # HVLA
			else: # low valence
				if(labels[i][1]>5): # high arousal
					em_labels.append(2) # LVHA
				else: # low arousal
					em_labels.append(3) # LVLA
		return em_labels

In [6]:
def kfold(allFeatures, allClasses, index):
    m = int(allClasses.shape[0])
    s = int(m/10)
    train_index = np.ones(m-s+1).astype(int)
    test_index  = np.ones(s).astype(int)
    for i in range(s*index,s*(index+1)):
        test_index[i-s*index] = i
    for i in range(0,s*index):
        train_index[i] = i
    for i in range(s + s*index,m):
        train_index[i-s] = i
    X_train = allFeatures.iloc[train_index]
    X_test = allFeatures.iloc[test_index]
    y_train = allClasses[train_index]
    y_test = allClasses[test_index]
    # normalize the xtrain data only with class labels
    return X_train, X_test, y_train, y_test

In [7]:
# Feature subset fitness function
def getFitness(individual, X_train, X_test, y_train, y_test):
    """
    parse our feature columns that we don't use
    apply one hot encoding to the features.
    """
    cols = [index for index in range(len(individual)) if individual[index] == 0]
    X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
    X_trainOhFeatures = pd.get_dummies(X_trainParsed)
    X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
    X_testOhFeatures = pd.get_dummies(X_testParsed)

    # Remove any columns that aren't in both the training and test sets
    sharedFeatures = set(X_trainOhFeatures.columns) & set(X_testOhFeatures.columns)
    removeFromTrain = set(X_trainOhFeatures.columns) - sharedFeatures
    removeFromTest = set(X_testOhFeatures.columns) - sharedFeatures
    X_trainOhFeatures = X_trainOhFeatures.drop(list(removeFromTrain), axis=1)
    X_testOhFeatures = X_testOhFeatures.drop(list(removeFromTest), axis=1)

    # Apply logistic regression on the data, and calculate accuracy
    clf = svm.SVC(kernel='poly').fit(X_trainOhFeatures, y_train)
    predictions = clf.predict(X_testOhFeatures)
    accuracy = accuracy_score(y_test, predictions)*100
    return (accuracy,)

In [8]:
def getFitness_all(individual, X_train, X_test, y_train, y_test):
    """
    Feature subset fitness function
    Parse our feature columns that we don't use
    Apply one hot encoding to the features.
    """
    cols = [index for index in range(len(individual)) if individual[index] == 0]
    #print(cols)
    #print(cols.shape)
    X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
    X_trainOhFeatures = pd.get_dummies(X_trainParsed)
    X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
    X_testOhFeatures = pd.get_dummies(X_testParsed)

    # Remove any columns that aren't in both the training and test sets
    sharedFeatures = set(X_trainOhFeatures.columns) & set(X_testOhFeatures.columns)
    removeFromTrain = set(X_trainOhFeatures.columns) - sharedFeatures
    removeFromTest = set(X_testOhFeatures.columns) - sharedFeatures
    X_trainOhFeatures = X_trainOhFeatures.drop(list(removeFromTrain), axis=1)
    X_testOhFeatures = X_testOhFeatures.drop(list(removeFromTest), axis=1)

    # Apply logistic regression on the data, and calculate accuracy
    clf = svm.SVC(kernel='poly').fit(X_trainOhFeatures, y_train)
    predictions = clf.predict(X_testOhFeatures)
    accuracy = accuracy_score(y_test, predictions)*100
    print("Accuracy Score: ", accuracy)
    try:
      print('Precision Score : ', precision_score(y_test, predictions)*100)
      print('Recall Score : ', recall_score(y_test, predictions)*100)
      print('F1 Score : ', f1_score(y_test, predictions)*100)
      print('Confusion Matrix : \n' + str(confusion_matrix(y_test, predictions)))
    except:
      pass
    return (accuracy, )

In [10]:
def getHof(toolbox):
    # Initialize variables to use eaSimple
    numPop = 100
    numGen = 50
    pop = toolbox.population(n=numPop)
    hof = tools.HallOfFame(numPop * numGen)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    # Launch genetic algorithm
    # change the crossover and mutation probability
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.65, mutpb=0.001, ngen=numGen, stats=stats, halloffame=hof, verbose=False)
    # Return the hall of fame
    return hof,log

In [11]:
def getMetrics(hof, X_train, X_test, y_train, y_test):
    # Get list of percentiles in the hall of fame
    percentileList = [i / (len(hof) - 1) for i in range(len(hof))]
    # Gather fitness data from each percentile
    testAccuracyList = []
    validationAccuracyList = []
    individualList = []
    for individual in hof:
        testAccuracy = individual.fitness.values
        validationAccuracy = getFitness(individual, X_train, X_test, y_train, y_test)
        testAccuracyList.append(testAccuracy[0])
        validationAccuracyList.append(validationAccuracy[0])
        individualList.append(individual)
    #testAccuracyList.reverse()
    #validationAccuracyList.reverse()
    return testAccuracyList, validationAccuracyList, individualList, percentileList

In [12]:
def drive(subject_name):
  dfData = pd.read_csv('/content/drive/MyDrive/Sequential methods for channel selection/our code/New/' + subject_name + '_arousal.csv')
  allFeatures = dfData
  names = allFeatures.columns
  scaler = MinMaxScaler()
  allFeatures = scaler.fit_transform(allFeatures)
  allFeatures = pd.DataFrame(allFeatures, columns=names)
  allFeatures = allFeatures.loc[:, allFeatures.apply(pd.Series.nunique) != 1]
  link = "/content/drive/MyDrive/Deap/" + subject_name + ".dat"
  with open(link, 'rb') as f:
    raw_data = pickle.load(f, encoding = 'latin1')
  labels = raw_data['labels']
  em_labels = emotion_label(labels, 'arousal') # get the emotion labels
  allClasses = np.array(em_labels)
  allFeatures, allClasses = shuffle(allFeatures, allClasses, random_state = 40)
  return allFeatures, allClasses

In [13]:
subject_names = ["s01", "s02", "s03", "s04", "s05", "s06", "s07", "s08", "s09", "s10", "s11", "s12",
                 "s13", "s14", "s15", "s16", "s17", "s18", "s19", "s20", "s21",
                "s22", "s23", "s24", "s25", "s26", "s27", "s28", "s29", "s30", "s31", "s32"]

In [14]:
def main_code(allFeatures, allClasses):
  for i in range(0, 10):
    print("Fold value = ", i+1)
    print("-"*40)
    #call k fold
    X_train, X_test, y_train, y_test = kfold(allFeatures, allClasses, i)
    #==========================    DEAP GLOBAL VARIABLES (viewable by SCOOP)      ======================
    # Create Individual
    creator.create("FitnessMax", base.Fitness, weights = (1.0,))
    creator.create("Individual", list, fitness = creator.FitnessMax)

    # Create Toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(allFeatures.columns) - 1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Continue filling toolbox...
    toolbox.register("evaluate", getFitness, X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb = 0.05)
    toolbox.register("select", tools.selTournament, tournsize = 7)
    #===================================================================================================
    
    # First, we will apply logistic regression using all the features to acquire a baseline accuracy.
    individual = [1 for i in range(len(allFeatures.columns))]
    Accuracy = getFitness_all(individual, X_train, X_test, y_train, y_test)
    if(Accuracy[0]==100):
      continue
    #print('\nAccuracy with all features from SVM: \t', Accuracy)
    # Now, we will apply a genetic algorithm to choose a subset of features that gives a better accuracy than the baseline.
    hof,log = getHof(toolbox)
    testAccuracyList, validationAccuracyList, individualList, percentileList = getMetrics(hof, X_train, X_test, y_train, y_test)
    # Get a list of subsets that performed best on validation data
    maxValAccSubsetIndicies = [index for index in range(len(validationAccuracyList)) if validationAccuracyList[index] == max(validationAccuracyList)]
    maxValIndividuals = [individualList[index] for index in maxValAccSubsetIndicies]
    maxValSubsets = [[list(allFeatures)[index] for index in range(len(individual)) if individual[index] == 1] for individual in maxValIndividuals]
    acc_list, prec_list, recall_list, f1_list, nooffeatures, conf_mat_list = [], [], [], [], [], []
    print('Optimal Feature Subset(s):')
    for i in range(len(maxValAccSubsetIndicies)):
        nooffeatures.append(len(maxValSubsets[i]))
        x_train = X_train[maxValSubsets[i]]
        x_test = X_test[maxValSubsets[i]]
        # knn implement ga-mlp for this
        #ga_mlp = ag_mlp(x_train , y_train , x_test, y_test, num_epochs = 20, size_mlp = 60, prob_mut = 0.001)
        #y_pred_val = ga_mlp.predict(x_test)
        try:
          clf = svm.SVC(kernel = 'poly').fit(x_train, y_train)
          y_pred_val = clf.predict(x_test)
          acc_list.append(accuracy_score(y_test, y_pred_val)*100)
        except:
          pass
        try:
          #print('Number Features In Subset: \t' + str(len(maxValSubsets[i])) + " ,accuracy: "+str(acc_list[i]))
          prec_list.append(precision_score(y_test,y_pred_val)*100)
          recall_list.append(recall_score(y_test,y_pred_val)*100)
          f1_list.append(f1_score(y_test, y_pred_val)*100)
          conf_mat_list.append(confusion_matrix(y_test, y_pred_val))
        except:
          pass
          # store the accuracy, precission and recall in a list then for each fold
    # convert list to array
    acc_list = np.array(acc_list)
    try:
      prec_list = np.array(prec_list)
      recall_list = np.array(recall_list)
      f1_list = np.array(f1_list)
      conf_mat_list = np.array(conf_mat_list)
      nooffeatures = np.array(nooffeatures)
    except:
      pass
    # find the maximum accuracy
    result = np.where(acc_list == np.amax(acc_list))[0] # now result has all the highest accuracy index
    # check for the precission
    temp_acc = acc_list[result]
    try:
      temp_prec = prec_list[result]
      temp_recall = recall_list[result]
      temp_f1 = f1_list[result]
      temp_conf_mat_list = conf_mat_list[result]
      temp_nooffeatures = nooffeatures[result]
    except:
      continue
    # find max in prec
    result_prec = np.where(temp_prec == np.amax(temp_prec))[0] # stores the index
    # find len of the result_prec
    if (len(result_prec)==1):
      print("Accuracy Score from SVM", temp_acc[result_prec])
      try:
        print('Precision Score : ', temp_prec[result_prec][0])
        print('Recall Score : ', temp_recall[result_prec][0])
        print('F1 Score : ', temp_f1[result_prec][0])
        print('Confusion Matrix : \n', temp_conf_mat_list[result_prec][0])
        print('No of features: ', temp_nooffeatures[result_prec][0])
      except:
        pass
    else:
      temp2_acc = temp_acc[result_prec]
      try:
        temp2_prec = temp_prec[result_prec]
        temp2_recall = temp_recall[result_prec]
        temp2_f1 = temp_f1[result_prec]
        temp2_conf_mat_list = temp_conf_mat_list[result_prec]
        temp2_nooffeatures = temp_nooffeatures[result_prec]
        # find the min features in the solution space
      except:
        pass
      final_index = np.where(temp2_nooffeatures == np.amin(temp2_nooffeatures))[0]
      print("Accuracy Score from SVM:", temp_acc[final_index][0])
      try:
        print('Precision Score : ', temp_prec[final_index][0])
        print('Recall Score : ', temp_recall[final_index][0])
        print('F1 Score : ', temp_f1[final_index][0])
        print('Confusion Matrix : \n', temp_conf_mat_list[final_index][0])
        print('No of features: ', temp_nooffeatures[final_index][0])
      except:
        pass
    """
    If multiple solution exists then find which one has the highest precission and that's all
    If there exist multiple solutions which covers those two condition then find which has the minimum no of features in the solution.
    """
    print("-"*40)

In [ ]:
for i in range(18, 32):
  print('-'*100)
  print("Subject No: ", subject_names[i])
  print('-'*100)
  allFeatures, allClasses = drive(subject_names[i])
  main_code(allFeatures, allClasses)

----------------------------------------------------------------------------------------------------
Subject No:  s19
----------------------------------------------------------------------------------------------------
Fold value =  1
----------------------------------------
Accuracy Score:  50.0
Precision Score :  0.0
Recall Score :  0.0
F1 Score :  0.0
Confusion Matrix : 
[[2 2]
 [0 0]]
Optimal Feature Subset(s):
Accuracy Score from SVM: 50.0
Precision Score :  0.0
Recall Score :  0.0
F1 Score :  0.0
Confusion Matrix : 
 [[2 2]
 [0 0]]
No of features:  794
----------------------------------------
Fold value =  2
----------------------------------------
Accuracy Score:  75.0
Precision Score :  100.0
Recall Score :  66.66666666666666
F1 Score :  80.0
Confusion Matrix : 
[[1 0]
 [1 2]]
Optimal Feature Subset(s):
Accuracy Score from SVM: 75.0
Precision Score :  100.0
Recall Score :  66.66666666666666
F1 Score :  80.0
Confusion Matrix : 
 [[1 0]
 [1 2]]
No of features:  813
--------------

In [15]:
for i in range(29, 32):
  print('-'*100)
  print("Subject No: ", subject_names[i])
  print('-'*100)
  allFeatures, allClasses = drive(subject_names[i])
  main_code(allFeatures, allClasses)

----------------------------------------------------------------------------------------------------
Subject No:  s30
----------------------------------------------------------------------------------------------------
Fold value =  1
----------------------------------------
Accuracy Score:  75.0
Precision Score :  0.0
Recall Score :  0.0
F1 Score :  0.0
Confusion Matrix : 
[[3 0]
 [1 0]]
Optimal Feature Subset(s):
Accuracy Score from SVM: 100.0
Precision Score :  100.0
Recall Score :  100.0
F1 Score :  100.0
Confusion Matrix : 
 [[3 0]
 [0 1]]
No of features:  256
----------------------------------------
Fold value =  2
----------------------------------------
Accuracy Score:  100.0
Precision Score :  100.0
Recall Score :  100.0
F1 Score :  100.0
Confusion Matrix : 
[[4]]
Fold value =  3
----------------------------------------
Accuracy Score:  75.0
Precision Score :  0.0
Recall Score :  0.0
F1 Score :  0.0
Confusion Matrix : 
[[3 1]
 [0 0]]
Optimal Feature Subset(s):
Accuracy Score f

**Accuracy**

In [30]:
accuracy = [95, 97.5, 85, 85, 80, 82.5, 87.5, 85, 87.5, 87.5, 97.5, 95, 97.5, 87.5, 95, 90, 95, 95, 92.5, 90, 87.5, 97.5, 95, 92.5, 95, 92.5, 90, 85, 92.5, 90, 95, 95]
print(len(accuracy))
accuracy = np.array(accuracy)
print(np.mean(accuracy))
print(np.std(accuracy))

32
91.09375
4.757292921136978


**Precision**

In [31]:
precision = [95, 95, 85, 91.667, 90, 80, 90, 78.334, 95, 90.833, 100, 95, 97.5, 88.333, 100, 100, 94.167, 96.667, 91.667, 89.1667, 86.667, 100, 91.667, 90, 88.333, 95, 89.167, 83.333, 90, 100, 94.167, 94.167]
print(len(precision))
precision = np.array(precision)
print(np.mean(precision))
print(np.std(precision))

32
92.057365625
5.519772401932063


**Recall or sensitivity**

In [32]:
recall = [100, 95, 85, 86.667, 76.667, 95, 91.667, 100, 88.333, 91.667, 97.5, 100, 100, 96.667, 91.667, 83.333, 100, 97.5, 100, 100, 100, 97.5, 100, 100, 97.5, 93.333, 100, 100, 100, 87.5, 100, 100]
recall = np.array(recall)
print(np.mean(recall))
np.std(recall)

95.39065625


6.063096381023968

**f1-Score**

In [29]:
f1 = [96.667, 96.0476, 89.238, 86.476, 78, 84.238, 88.476, 86.476, 89.333, 89.238, 98.571, 97.143, 98.571, 91.143, 94.667, 88.333, 96.571, 96.571, 95.143, 93.714, 92.286, 98.571, 94.667, 93, 91.238, 92.667, 93.714, 89.81, 94, 92.381, 96.571, 96.571]
f1 = np.array(f1)
print(np.mean(f1))
print(np.std(f1))

92.50289375
4.599280238522212
